In [6]:
import pandas as pd
from calendar import monthrange
from math import ceil
import numpy as np
import datetime as dt

data = pd.read_csv('./data_input_nozerofill_2018.csv', sep=',', delimiter=',')
data_model = data.copy()

#usando apenas as top 100 linhas (linhas com mais exemplos)
top100_linhalist = data_model.linha.value_counts().index[:100]

In [12]:
## Inserir zero rows
def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """

    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))

# linha_list = data_model.linha.unique()


for linha in top100_linhalist:
    currentDataModel_Linha = data_model.loc[data_model.linha == linha]
    mes_max = currentDataModel_Linha.mes.max()
    mes_min = currentDataModel_Linha.mes.min()
    for mes in range(mes_min, mes_max+1):
        currentDataModel_Mes = currentDataModel_Linha.loc[currentDataModel_Linha.mes == mes]
        dia_max = monthrange(2020, mes)[1]
        for dia in range(1, dia_max+1):
            currentDataModel_Dia = currentDataModel_Mes.loc[currentDataModel_Mes.d_mes == dia]
            
            for hora in range(0, 24):
                currentDataModel_Hora = currentDataModel_Dia[currentDataModel_Dia.hora == hora]
                if currentDataModel_Hora.empty == False:
                    continue
                else:
                    # print('INSERINDO NO MES '+str(mes)+' NO DIA '+str(dia)+ ' NA HORA '+str(hora))
                    #2018-01-01 00:00:00
                    if (mes == 2 and dia > 28):
                        continue
                    dd = dt.datetime.strptime('2018-'+str(mes)+'-'+str(dia)+' '+str(hora)+':00:00', "%Y-%m-%d %H:00:00")
                    new_row = {
                            'linha': linha,
                            'data_hora': dd,
                            'validations_per_hour': 0,
                            'd_semana': dd.weekday(),
                            'hour_sin': np.sin(2 * np.pi * hora/23.0),
                            'hour_cos': np.cos(2 * np.pi * hora/23.0),
                            'hora': hora,
                            'd_mes': dia,
                            'd_ano': dd.timetuple().tm_yday,
                            'mes': mes,
                            'semana_do_mes': week_of_month(dd)
                        }
                    # print(new_row)
                    data_model = data_model.append(new_row, ignore_index=True)

                #insert validation zero
data_model.loc[data_model.mes == 3]

,linha,data_hora,validations_per_hour,d_semana,hour_sin,hour_cos,hora,d_mes,d_ano,mes,semana_do_mes
315037,120,2018-03-01 00:00:00,1,3,0.000000,1.000000,0,1,60,3,0
315058,122,2018-03-01 00:00:00,1,3,0.000000,1.000000,0,1,60,3,0
315178,19,2018-03-01 00:00:00,2,3,0.000000,1.000000,0,1,60,3,0
315257,21,2018-03-01 00:00:00,1,3,0.000000,1.000000,0,1,60,3,0
315457,220,2018-03-01 00:00:00,4,3,0.000000,1.000000,0,1,60,3,0
...,...,...,...,...,...,...,...,...,...,...,...
1247136,339,2018-03-30 03:00:00,0,4,0.730836,0.682553,3,30,89,3,5
1247137,339,2018-03-31 00:00:00,0,5,0.000000,1.000000,0,31,90,3,5
1247138,339,2018-03-31 01:00:00,0,5,0.269797,0.962917,1,31,90,3,5
1247139,339,2018-03-31 02:00:00,0,5,0.519584,0.854419,2,31,90,3,5


In [10]:
mes

2

In [ ]:
data_model = data_model.sort_values(['linha', 'data_hora'], ascending=[True, True])

In [ ]:
data_model.to_csv('./data_input_zerofill_2018.csv', index=False, sep=';')

In [ ]:
data_model

,linha,data_hora,validations_per_hour,d_semana,hour_sin,hour_cos,hora,d_mes,d_ano,mes,semana_do_mes
1150212,1,2020-01-11 02:00:00,0,5,5.195840e-01,0.854419,2,11,11,1,2
1150213,1,2020-01-31 03:00:00,0,4,7.308360e-01,0.682553,3,31,31,1,5
1150214,1,2020-02-02 03:00:00,0,6,7.308360e-01,0.682553,3,2,33,2,1
1150215,1,2020-02-19 02:00:00,0,2,5.195840e-01,0.854419,2,19,50,2,4
1150216,1,2020-02-20 02:00:00,0,3,5.195840e-01,0.854419,2,20,51,2,4
...,...,...,...,...,...,...,...,...,...,...,...
995001,999,2018-07-02 21:00:00,5,0,-5.195840e-01,0.854419,21,2,183,7,1
995002,999,2018-07-02 22:00:00,8,0,-2.697968e-01,0.962917,22,2,183,7,1
995003,999,2018-07-02 23:00:00,1,0,-2.449294e-16,1.000000,23,2,183,7,1
1144682,999,2018-07-30 20:00:00,2,0,-7.308360e-01,0.682553,20,30,211,7,5
